In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [ ]:
#generator and discriminator

# Define the Generator network
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.fc(x)
        return x

# Define the Discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.fc(x)
        return x



In [ ]:
input_dim = 100
output_dim = 784
hidden_dim = 128
lr = 0.00002
batch_size = 64
num_epochs = 50

In [ ]:

# Create instances of the generator and discriminator networks
generator = Generator(input_dim, output_dim, hidden_dim)
discriminator = Discriminator(output_dim, hidden_dim)


In [ ]:
# Define the loss function (BCELoss) and the optimizers (Adam) for generator and discriminator
criterion = nn.BCELoss()
generator_optimizer = optim.Adam(generator.parameters(), lr=lr)
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=lr)

In [ ]:
# Load the Fashion MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(
    root='path/to/dataset',
    train=True,
    download=True,
    transform=transform
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
# Generate new samples
num_samples = 10
noise = torch.randn(num_samples, input_dim)
generated_images = generator(noise)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    generator.train()
    discriminator.train()

    for batch_idx, (real_images, _) in enumerate(train_loader):
        batch_size = real_images.size(0)
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        # Train the discriminator
        discriminator_optimizer.zero_grad()

        real_outputs = discriminator(real_images.view(batch_size, -1))
        real_loss = criterion(real_outputs, real_labels)

        noise = torch.randn(batch_size, input_dim)
        fake_images = generator(noise)
        fake_outputs = discriminator(fake_images.detach())
        fake_loss = criterion(fake_outputs, fake_labels)

        discriminator_loss = real_loss + fake_loss
        discriminator_loss.backward()
        discriminator_optimizer.step()

        # Train the generator
        generator_optimizer.zero_grad()

        fake_outputs = discriminator(fake_images)
        generator_loss = criterion(fake_outputs, real_labels)

        generator_loss.backward()
        generator_optimizer.step()

    # Print progress and losses
    print(f"Epoch [{epoch+1}/{num_epochs}], Generator Loss: {generator_loss.item():.4f}, Discriminator Loss: {discriminator_loss.item():.4f}")


In [ ]:
#@title
import pickle

# Specify the file path for saving the generator model
generator_path = 'generator_model.pkl'

# Save the generator model
with open(generator_path, 'wb') as f:
    pickle.dump(generator, f)
